In [2]:
import sys
import os

# Добавьте путь к корню вашего проекта
project_path = "/home/andrey/PycharmProjects/ProtoLLM/"
sys.path.append(project_path)

In [3]:
import chromadb
import uuid
from dotenv import load_dotenv
from langchain_community.embeddings.huggingface_hub import HuggingFaceHubEmbeddings
from protollm_sdk.models.job_context_models import PromptModel
from protollm_sdk.jobs.outer_llm_api import OuterLLMAPI
from protollm.rags.rag_core.retriever import DocRetriever, DocsSearcherModels
from definitions import CONFIG_PATH

Функция для подключения к ChromaDB

In [4]:
def init_chroma_client():
    host, port = os.environ.get("CHROMA_DEFAULT_SETTINGS").split(':')
    return chromadb.HttpClient(
        host=host,
        port=int(port),
        settings=chromadb.Settings(),
    )

Функция для запроса к ChromaDB

In [5]:
def proto_view(
    query: str,
    collection: str,
    k: int = 1,
    embedding_function: HuggingFaceHubEmbeddings = None,
) -> list:
    # Возвращает k ближайших к запросу фрагментов
    embedding_host = os.environ.get("EMBEDDING_HOST")
    embedding_function = HuggingFaceHubEmbeddings(model=embedding_host)
    chroma_client = init_chroma_client()

    docs_searcher_models = DocsSearcherModels(embedding_model=embedding_function, chroma_client=chroma_client)
    retriever = DocRetriever(top_k=k,
                             docs_searcher_models=docs_searcher_models,
                             )

    return retriever.retrieve_top(collection_name=collection, query=query)

Функция для запроса к API VseGPT

In [6]:
def outer_llm(question: str,
              meta: dict,
              key: str,
              model: str):
    llmapi = OuterLLMAPI(key)
    llm_request = PromptModel(job_id=str(uuid.uuid4()),
                              meta=meta,
                              content=question)
    res = llmapi.inference(llm_request)
    return res.content

In [7]:
load_dotenv(CONFIG_PATH)

# Настройки БЯМ
model = "openai/gpt-4o-2024-08-06"
meta = {"temperature": 0.05,
        "tokens_limit": 4096,
        "stop_words": None}
key = os.environ.get("VSE_GPT_KEY")

# Название коллекции в БД
collection_name = "strategy-spb"

# Вопрос
question = 'Какие задачи Стратегия ставит в области энергосбережения?'

# Извлечение контекста из БД
context = proto_view(question, collection_name)
context = f'Вопрос: {question} Контекст: {context[0].page_content}'

# Получение ответа от БЯМ
print(f'Ответ VseGPT LLM: \n {outer_llm(context, meta, key, model)}')

/tmp/ipykernel_64625/1530686652.py:9: LangChainDeprecationWarning: The class `HuggingFaceHubEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpointEmbeddings``.
  embedding_function = HuggingFaceHubEmbeddings(model=embedding_host)
/home/andrey/PycharmProjects/ProtoLLM/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Ответ VseGPT LLM: 
 Стратегия в области энергосбережения ставит задачи, направленные на развитие энергосбережения и повышение энергетической эффективности систем коммунальной инфраструктуры. Это включает в себя:

1. Реализацию организационных, правовых, технических, технологических и экономических мер, которые способствуют уменьшению объема используемых энергетических ресурсов при сохранении их полезного эффекта.
2. Развитие альтернативных источников энергии.
3. Оптимизацию процессов технологического присоединения к инженерным сетям, что включает минимизацию административных процедур и требуемых документов.
4. Обеспечение соблюдения единых стандартов обслуживания и ускорение процессов подключения к сетям инженерной инфраструктуры, таких как электро-, тепло-, газо-, водоснабжение и водоотведение.

Эти меры направлены на повышение энергоэффективности и снижение энергопотребления в коммунальной инфраструктуре.
